## Яндекс Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:

* Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
* Петербург — культурная столица, со своими вкусами.

На данных Яндекс Музыки необходимо сравнить поведение пользователей сервиса "Яндекс Музыка" двух столиц.

<b>Цель исследования</b> заключается в проверке трёх гипотез:

1. Активность пользователей зависит от дня недели, причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

<b>Ход исследования</b>

Данные о поведении пользователей представлены файлом `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез требуется провести обзор данных, в том числе проверить данные на ошибки и оценить их влияние на исследование. Затем, на этапе предобработки необходимо рассмотреть возможность исправить самые критичные ошибки данных. Таким образом, исследование пройдёт в четыре этапа:

1. Обзор данных.
2. Предобработка данных.
<br>2.1. Стиль заголовков
<br>2.2. Проверка на пропуски значений
<br>2.3. Проверка на дубликаты
3. Проверка гипотез.
<br>3.1. Сравнение поведения пользователей двух столиц
<br>3.2. Музыка в начале и в конце недели
<br>3.3. Жанровые предпочтения в Москве и Петербурге
4. Подведение итогов.

### 1. Обзор данных

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# импорт данных и сохранение в df
df = pd.read_csv('yandex_music_project.csv')

In [3]:
# получение первых 5 строк датасета
df.head()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [4]:
# получение общей информации о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице 7 столбцов. Тип данных во всех столбцах — `object`. Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения. Также, в наименованиях признаков видны нарушения стиля:

* Строчные буквы сочетаются с прописными.
* Встречаются пробелы.
* Не применен так называемый `змеиный регистр` наименования признаков 

**Выводы:** в каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем. Чтобы двигаться дальше, нужно устранить проблемы в данных.

### 2. Предварительная обработка данных 

#### 2.1.  Стиль заголовков

In [5]:
# перечень признаков
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [6]:
# переименование признаков
df = df.rename(columns={'  userID': 'user_id', 
                        'Track': 'track', 
                        '  City  ': 'city', 
                        'Day': 'day'})

In [7]:
# проверка результатов переименования
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

#### 2.2. Проверка на пропуски значений

In [8]:
# подсчёт пропусков
df.isnull().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так, в `track` и `artist` пропуски не важны для исследования. Достаточно заменить их явными обозначениями. Но пропуски в признаке `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в данном проекте, поэтому мы заполним эти пропуски заглушками и оценим, насколько они могут вопредить расчётам.

In [9]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

In [10]:
# проверка количества пропусков
df.isnull().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

#### 2.3. Проверка на дубликаты

In [11]:
# подсчёт явных дубликатов
df.duplicated().sum()

3826

In [12]:
# удаление явных дубликатов
df = df.drop_duplicates()

In [13]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Теперь необходимо избавиться от неявных дубликатов в признаке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

In [14]:
# уникальные названия жанров
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

По итогам ознакомления с уникальными признаками нами были установлены неявные дубликаты жанра `hiphop`. Это названия с ошибками или альтернативные названия того же жанра: `hip`,`hop`, `hip-hop`. Чтобы очистить от них таблицу применим метод `replace()` с двумя аргументами: списком строк-дубликатов (включащий `hip`, `hop` и `hip-hop`) и строкой с правильным значением. Необходимо исправить признак `genre`: заменить каждое значение из списка дубликатов на верное.

In [15]:
# очистка неявных дубликатов
duplicates = ['hip', 'hop', 'hip-hop'] # дублирующие значения
name = 'hiphop' # правильное значение
df['genre'] = df['genre'].replace(duplicates, name)

In [16]:
# проверка результатов очистки неявных дубликатов
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы:** предобработка обнаружила три проблемы в данных: 1) нарушения в стиле заголовков; 2) пропущенные значения; 3) явные и неявные дубликаты. Нами были исправлены заголовки, чтобы упростить работу с данными. Без дубликатов исследование станет более точным. Пропущенные значения были заменены на заглушку `unknown`. Предстоит рассмотреть потенциальную угрозу для исследования пропусков по признаку `genre`.

### 3. Проверка гипотез

#### 3.1.  Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверьте это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого необходимо разделить пользователей Москвы и Санкт-Петербурга. И затем сравнить, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

In [17]:
# подсчёт прослушиваний в каждом городе
df.groupby('city')['user_id'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Однако, из этого не следует, что московские пользователи чаще слушают музыку, просто самих пользователей в Москве больше.

In [18]:
# подсчёт прослушиваний в каждый из трёх дней
df.groupby('day')['day'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: day, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [19]:
# функция для подсчёта прослушиваний для конкретного города и дня

# с помощью последовательной фильтрации с логической индексацией она 
# сначала получит из исходных данных строки с нужным днём,
# затем из результата отфильтрует строки с нужным городом,
# методом count() посчитает количество значений в колонке user_id,
# это количество функция вернёт в качестве результата

def number_tracks(day, city):
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Применим функцию `number_tracks()` шесть раз, меняя значение параметров так, чтобы получить данные для каждого города в каждый из трёх дней.

In [20]:
# количество прослушиваний в Москве по понедельникам
number_tracks('Monday', 'Moscow')

15740

In [21]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks('Monday', 'Saint-Petersburg')

5614

In [22]:
# количество прослушиваний в Москве по средам
number_tracks('Wednesday', 'Moscow')

11056

In [23]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [24]:
# количество прослушиваний в Москве по пятницам
number_tracks('Friday', 'Moscow')

15945

In [25]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks('Friday', 'Saint-Petersburg')

5895

Объединим полученные данные подсчётов в одном датафрейме.

In [26]:
# сводная таблица с результатами
data = ['Moskow', 15740, 11056, 15945], ['Saint-Petersburg', 5614, 7003, 5895]
columns = ['city', 'monday', 'wednesday', 'friday']
pivotcity = pd.DataFrame(data=data, columns=columns)
pivotcity

,city,monday,wednesday,friday
0,Moskow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы:** полученные нами данные расчётов показывают наглядую разницу поведения пользователей в двух столицах. Так, в Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад. В Петербурге наоборот, больше слушают музыку по средам, при этом активность в понедельник и пятницу здесь почти в равной мере уступает среде. Следовательно, данные говорят в пользу первой гипотезы.

#### 3.2.  Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. Создадим два датафрейма по разным городам.

In [27]:
# срез по жителям Москвы 
moscow_general = df[df['city'] == 'Moscow']

In [28]:
# срез по жителям Санкт-Петербурга
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:

1. таблица (датафрейм) с данными,
2. день недели,
3. начальная временная метка в формате 'hh:mm',
4. последняя временная метка в формате 'hh:mm'.

Функция должна вернуть нам информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [29]:
def genre_weekday(df, day, time1, time2):
    # последовательная фильтрация
    # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day'] == day]
    # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time'] < time2]
    # оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time'] > time1]
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров
    #возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00).

In [30]:
# вызов функции для утра понедельника в Москве
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [31]:
# вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [32]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [33]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы:** Относительно топ-10 жанров в понедельник утром мы можем утверждать, что в Москве и Санкт-Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр `world`, а в петербургский — `джаз` и `классика`. В Москве пропущенных значений оказалось так много, что значение `unknown` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования. Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Следовательно, вторая гипотеза подтвердилась лишь частично. Так, пользователи слушают похожую музыку в начале недели и в конце. Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз. Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные данные о жанрах.

#### 3.3  Жанровые предпочтения в Москве и Петербурге

Гипотеза о даных звучит следующим образом: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве. А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [34]:
# одной строкой: группировка таблицы moscow_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [38]:
# просмотр топ-10 жанров по Москве
moscow_genres[:10]

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [39]:
# одной строкой: группировка таблицы spb_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [40]:
# просмотр топ-10 жанров по Санкт-Петербургу
spb_genres[:10]

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Вывод:** гипотеза частично подтвердилась. Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка. Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.

### 4.  Итоги исследования

Нами были проверены три гипотезы относительно данных. По результатам проверки установлены следующие факты:

|№ п/п |Описание гипотезы|Результат проверки гипотезы|
|------|:----------------|:--------------------------|
|1.|Активность пользователей зависит от дня недели, причём в Москве и Петербурге это проявляется по-разному.|<b>Подтвердилась полностью.</b>|
|2.|В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.|<b>Подтвердилась частично.</b> Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам: в Москве слушают музыку жанра `world`, в Петербурге — `джаз` и `классику`. Этот результат мог оказаться иным, если бы не пропуски в данных.|
|3.|Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.|<b>Не подтвердилась.</b> Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.|